# Multimodal CTR Prediction (MM-CTR)

This notebook presents a complete end-to-end solution for the MM-CTR Challenge, covering both multimodal item embedding (Task 1) and multimodal CTR prediction (Task 2).


# 1. Introduction

This project addresses the Multimodal Click-through Rate (MM-CTR) Prediction Challenge at WWW 2025 EReL@MIR. The goal is to leverage multimodal item information—text and images—to improve click-through rate prediction under low-latency constraints. We implement the complete pipeline covering Task 1: Multimodal Item Embedding and Task 2: Multimodal CTR Prediction in a single, reproducible notebook.

In [ ]:
# Force a compatible protobuf version
# Some libraries (CLIP, sentence-transformers) may break with newer versions
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

In [ ]:
# Install OpenAI CLIP directly from GitHub
# Kaggle does not provide it by default
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# Additional libraries for training & evaluation
!pip install -q polars tqdm

In [ ]:
# ==========================
# Core imports
# ==========================
import os
import warnings

import numpy as np
import pandas as pd
import torch

from PIL import Image
import clip
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

import polars as pl
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import BinaryAUROC
from tqdm import tqdm


warnings.filterwarnings("ignore")

# 2. Dataset Overview

We use the MicroLens-1M dataset, which contains user click interactions and rich item content features such as titles and cover images. Public data are used for training and validation, while private data are reserved for testing.

In [ ]:
# ==========================
# Dataset paths (Kaggle)
# ==========================
INFO_FILE = "/kaggle/input/item_info.parquet"
FEATURE_FILE = "/kaggle/input/item_feature.parquet"
IMAGE_DIR = "/kaggle/input/item_images/item_images/"

SAVE_PATH = "/kaggle/working/item_info_multimodal_emb.parquet"

# ==========================
# Read parquet files
# ==========================
df_info = pd.read_parquet(INFO_FILE)
df_features = pd.read_parquet(FEATURE_FILE)

print("Item info shape :", df_info.shape)
print("Item feature shape :", df_features.shape)


# 3. Task 1: Multimodal Item Embedding
In this section, we extract textual and visual representations for each item, then fuse them into a compact 128-dimensional embedding.


# 3.1 Text Embedding and Image Embedding

We encode item textual information using a pretrained sentence-level encoder to obtain dense semantic representations.

Item cover images are processed using a pretrained vision model to extract visual embeddings.

In [ ]:
# ==========================
# Device configuration
# ==========================
compute_device = "cuda" if torch.cuda.is_available() else "cpu"

# ==========================
# Text encoder (Sentence-BERT)
# ==========================
text_encoder = SentenceTransformer("all-MiniLM-L6-v2")
text_encoder.eval()

# ==========================
# Image encoder (CLIP ResNet-50)
# ==========================
clip_net, clip_preprocess = clip.load("RN50", device=compute_device)
clip_net.eval()

In [ ]:
# ==========================
# Text embeddings
# ==========================
item_titles = df_features["item_title"].tolist()

text_vectors = text_encoder.encode(
    item_titles,
    batch_size=256,
    show_progress_bar=True
)

# ==========================
# Image embeddings 
# ==========================
def extract_image_embedding(image_path):
    """
    Compute CLIP embedding for a single image.
    If the image is missing or corrupted, return a zero vector.
    """
    try:
        img = Image.open(image_path).convert("RGB")
        img_tensor = clip_preprocess(img).unsqueeze(0).to(compute_device)

        with torch.no_grad():
            img_emb = clip_net.encode_image(img_tensor)

        return img_emb.cpu().numpy().squeeze()

    except Exception:
        return np.zeros(clip_net.visual.output_dim)


# ==========================
# Encode all item images
# ==========================
image_vectors = []

for iid in df_features["item_id"]:
    img_path = os.path.join(IMAGE_DIR, f"image{iid}.jpg")
    image_vectors.append(extract_image_embedding(img_path))

image_vectors = np.array(image_vectors)

print("Image embedding matrix shape:", image_vectors.shape)


In [ ]:
# Inspect available columns
print(df_info.columns.tolist())

# 3.3 Multimodal Fusion and Dimensionality Reduction

Text and image embeddings are concatenated to form multimodal representations. To satisfy competition constraints and reduce inference latency, PCA is applied to project embeddings to 128 dimensions.

In [ ]:
from IPython.display import FileLink

# ==========================
# Multimodal fusion (text + image)
# ==========================
multimodal_features = np.concatenate(
    [text_vectors, image_vectors],
    axis=1
)

print("Fused embedding shape:", multimodal_features.shape)

# ==========================
# Padding first row (alignment with item_info)
# ==========================
zero_padding = np.zeros((1, multimodal_features.shape[1]))
multimodal_features_padded = np.vstack(
    [zero_padding, multimodal_features]
)

print("After padding:", multimodal_features_padded.shape)

# ==========================
# Dimensionality reduction (PCA → 128)
# ==========================
pca_model = PCA(n_components=128)
multimodal_128d = pca_model.fit_transform(multimodal_features_padded)

print("Final embedding shape:", multimodal_128d.shape)

# ==========================
# Save into item_info
# ==========================
df_info["item_emb_d128"] = list(multimodal_128d)
df_info.to_parquet(SAVE_PATH, index=False)
#You can download the saved file from /kaggle/working/ directory
print(f"Saved fused embeddings to: {SAVE_PATH}")

In [ ]:
# Reload saved file for sanity check
df_check = pd.read_parquet(SAVE_PATH)

print(df_check.head())
print("Embedding dimension example:", len(df_check["item_emb_d128"].iloc[0]))

# 4. Task 2: Multimodal CTR Prediction

In this task, we focus on predicting the click-through rate (CTR) by effectively leveraging the multimodal item embeddings generated in Task 1. The objective is to design an end-to-end neural CTR model that integrates user interaction features with multimodal item representations.

The model takes as input user identifiers, target items, historical interaction sequences, and engagement signals (likes and views), together with the 128-dimensional multimodal item embeddings. These features are jointly learned through embedding layers and a multilayer perceptron (MLP) to estimate the probability of a user clicking on a given item.

The model is trained using binary cross-entropy loss and evaluated using the Area Under the ROC Curve (AUC), following the official data split and competition constraints.


# 4.1 Problem Formulation

Given user interaction history and item multimodal embeddings, the task is to predict the probability of a click (pCTR; I named it Task1&2 in the submission.csv as you'll find at the end of this notebook) for each user–item pair.

In [ ]:
# ==========================
# Kaggle dataset paths
# ==========================
BASE_PATH = "/kaggle/input/"  

train_df = pl.read_parquet(f"{BASE_PATH}/train.parquet")
valid_df = pl.read_parquet(f"{BASE_PATH}/valid.parquet")
test_df  = pl.read_parquet(f"{BASE_PATH}/test.parquet")

# Load item_info WITH multimodal embeddings
item_info = pl.read_parquet(
    "/kaggle/working/item_info_multimodal_emb.parquet"
)

print(
    "Train / Valid / Test shapes:",
    train_df.shape, valid_df.shape, test_df.shape
)

In [ ]:
# ==========================
# Device configuration
# ==========================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
# ==========================
# Build item embedding tensor (128-d multimodal)
# ==========================
item_emb_matrix = np.stack(
    item_info["item_emb_d128"].to_numpy()
)

item_emb_tensor = torch.tensor(
    item_emb_matrix,
    dtype=torch.float32,
    device=device
)

num_items = item_emb_tensor.shape[0]
print("Number of items (including padding):", num_items)


# 4.2 Model Architecture

We design an end-to-end neural CTR model that ingests user features and item multimodal embeddings. The model outputs a scalar pCTR via fully connected layers.

In [ ]:
# ==========================
# CTR Dataset
# ==========================
class CTRDataset(Dataset):
    """
    PyTorch Dataset for CTR prediction.
    Handles both train/validation and test modes.
    """

    def __init__(self, df, is_test=False):
        self.is_test = is_test

        self.user_id  = df["user_id"].to_numpy()
        self.item_id  = df["item_id"].to_numpy()
        self.item_seq = df["item_seq"].to_numpy()
        self.likes    = df["likes_level"].to_numpy()
        self.views    = df["views_level"].to_numpy()

        if not is_test:
            self.label = df["label"].to_numpy()
        else:
            self.ID = df["ID"].to_numpy()

    def __len__(self):
        return len(self.user_id)

    def __getitem__(self, idx):
        sample = {
            "user_id": torch.tensor(self.user_id[idx], dtype=torch.long),
            "item_id": torch.tensor(self.item_id[idx], dtype=torch.long),
            "item_seq": torch.tensor(self.item_seq[idx], dtype=torch.long),
            "likes": torch.tensor(self.likes[idx], dtype=torch.long),
            "views": torch.tensor(self.views[idx], dtype=torch.long),
        }

        if self.is_test:
            sample["ID"] = torch.tensor(self.ID[idx], dtype=torch.long)
        else:
            sample["label"] = torch.tensor(self.label[idx], dtype=torch.float32)

        return sample


In [ ]:
# ==========================
# DataLoaders 
# ==========================
BATCH_SIZE = 1024

train_loader = DataLoader(
    CTRDataset(train_df),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,      # IMPORTANT for Kaggle users
    pin_memory=False    # IMPORTANT for Kaggle users
)

valid_loader = DataLoader(
    CTRDataset(valid_df),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False
)

test_loader = DataLoader(
    CTRDataset(test_df, is_test=True),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False
)


In [ ]:
# ==========================
# MMCTR Model
# ==========================
class MMCTRModel(nn.Module):
    """
    Multimodal CTR model combining:
    - User embedding
    - Item embedding
    - Sequential item history
    - Context features (likes, views)
    - Precomputed multimodal item embeddings (CLIP + SBERT)
    """

    def __init__(self, num_users, num_items, emb_dim=32):
        super().__init__()

        self.user_emb = nn.Embedding(num_users + 1, emb_dim, padding_idx=0)
        self.item_emb = nn.Embedding(num_items, emb_dim, padding_idx=0)

        self.likes_emb = nn.Embedding(11, emb_dim)
        self.views_emb = nn.Embedding(11, emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(emb_dim * 5 + 128, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, user_id, item_id, item_seq, likes, views):
        user_e = self.user_emb(user_id)
        item_e = self.item_emb(item_id)

        # Sequence pooling (masked mean)
        seq_e = self.item_emb(item_seq)
        mask = (item_seq != 0).unsqueeze(-1)
        seq_e = (seq_e * mask).sum(1) / (mask.sum(1) + 1e-8)

        likes_e = self.likes_emb(likes)
        views_e = self.views_emb(views)

        # Multimodal item embedding (frozen)
        item_mm_e = item_emb_tensor[item_id]

        x = torch.cat(
            [user_e, item_e, seq_e, likes_e, views_e, item_mm_e],
            dim=1
        )

        return self.mlp(x).squeeze(1)


In [ ]:
# ==========================
# Model setup
# ==========================
num_users = int(train_df["user_id"].max())

model = MMCTRModel(
    num_users=num_users,
    num_items=num_items
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

auc_metric = BinaryAUROC().to(device)

# 4.3 Model Training and Validation

We train the multimodal CTR model on the training set and monitor its performance on the validation set using AUC. Early stopping is applied to prevent overfitting, and the best-performing model is saved for final inference.


In [ ]:
# ==========================
# Training loop
# ==========================
EPOCHS = 20
best_valid_auc = 0.0
patience = 3
patience_counter = 0

MODEL_PATH = "/kaggle/working/best_mmctr_model.pt"

for epoch in range(EPOCHS):

    # ===== TRAIN =====
    model.train()
    epoch_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()

        logits = model(
            batch["user_id"].to(device),
            batch["item_id"].to(device),
            batch["item_seq"].to(device),
            batch["likes"].to(device),
            batch["views"].to(device)
        )

        loss = criterion(logits, batch["label"].to(device))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1} | Train Loss: {epoch_loss / len(train_loader):.4f}")

    # ===== VALID =====
    model.eval()
    auc_metric.reset()

    with torch.no_grad():
        for batch in valid_loader:
            logits = model(
                batch["user_id"].to(device),
                batch["item_id"].to(device),
                batch["item_seq"].to(device),
                batch["likes"].to(device),
                batch["views"].to(device)
            )
            probs = torch.sigmoid(logits)
            auc_metric.update(probs, batch["label"].to(device))

    valid_auc = auc_metric.compute().item()
    print(f"Epoch {epoch+1} | Valid AUC: {valid_auc:.4f}")

    # ===== EARLY STOPPING =====
    if valid_auc > best_valid_auc:
        best_valid_auc = valid_auc
        patience_counter = 0
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"🔥 New best model saved (AUC = {best_valid_auc:.4f})")
    else:
        patience_counter += 1
        print(f"⏳ No improvement ({patience_counter}/{patience})")

    if patience_counter >= patience:
        print("🛑 Early stopping triggered")
        break

# Experimental Results

The proposed multimodal CTR model achieves competitive performance on the validation set, demonstrating the effectiveness of incorporating multimodal item embeddings into CTR prediction. The final predictions are submitted to the Codabench platform for official evaluation.


In [ ]:
print(f"Best Validation AUC: {best_valid_auc:.4f}")

# 5. Test Inference and Submission

In this section, we load the best trained model and generate predicted click-through rates (pCTR) for the test set. The predictions are formatted according to the Codabench submission requirements.


In [ ]:
# ==========================
# Load best trained model
# ==========================
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

test_ids = []
test_pctr = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Inference on test"):
        logits = model(
            batch["user_id"].to(device),
            batch["item_id"].to(device),
            batch["item_seq"].to(device),
            batch["likes"].to(device),
            batch["views"].to(device)
        )

        probs = torch.sigmoid(logits)

        test_ids.extend(batch["ID"].cpu().numpy())
        test_pctr.extend(probs.cpu().numpy())

In [ ]:
# ==========================
# Build submission file
# ==========================
import pandas as pd

submission_df = pd.DataFrame({
    "ID": test_ids,
    "Task1&2": test_pctr
})

SUB_PATH = "/kaggle/working/submission.csv"
submission_df.to_csv(SUB_PATH, index=False)

print("Submission file saved at:", SUB_PATH)


In [ ]:
assert len(submission_df) == test_df.shape[0]
assert submission_df["Task1&2"].between(0, 1).all()
assert not submission_df.isnull().any().any()
assert submission_df["ID"].is_unique

print("✅ Submission sanity check passed")

## 7. Conclusion
We implemented a complete multimodal CTR prediction pipeline, combining textual and visual item information with user interaction data. The proposed model improves CTR prediction performance and satisfies the low-latency and reproducibility constraints of the MM-CTR Challenge.